# Решение с помощью **BERT**

## Библиотеки (здесь не шуметь)

In [515]:
import numpy as np
import pandas as pd
import torch
import transformers
import tqdm
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score
from deeppavlov.core.common.file import read_json
from deeppavlov import build_model, configs
from transformers import BertTokenizer, BertModel,BertConfig
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier

In [443]:
#pip install catboost


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [444]:
#pip install ipywidgets


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#pip install transformers

In [446]:
#pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Загрузка датасета

In [447]:
df_tweets=pd.read_csv('/content/drive/MyDrive/Bert/toxic_comments.csv')

В датасете имеется дисбаланс классов. Исправлю его сравняв ответы 0 и 1 путем DawnSampling. Так же, посколько в конфигах BERT'а установлено 512 максимум фичей на вход, но выбрав часть выборки для расчета, я выберу те твиты, которые не длиннее 512 символов.

In [448]:

print(df_tweets['toxic'].mean())
df_tweets['lenght']=df_tweets['text'].str.len()
#df_tweets = df_tweets.sample(200).reset_index(drop=True) 
df_tweets = df_tweets[df_tweets['lenght']<=512]
df_tweets[df_tweets['toxic'] == 0] = df_tweets[df_tweets['toxic']== 0].sample(frac=0.125, random_state=42)
df_tweets = df_tweets.dropna()
df_tweets=df_tweets.sample(500).reset_index(drop=True) 

0.10161213369158527


In [449]:
print(df_tweets['toxic'].mean())

0.498


In [450]:
df_tweets['toxic'].isna().mean()

0.0

In [451]:
df_tweets

,Unnamed: 0,text,toxic,lenght
0,30990.0,Take a look at the page now- an actual Douche ...,1.0,477.0
1,84200.0,"""\n\nWhimsical?\n\nThought that """"whimsical"""" ...",0.0,217.0
2,118023.0,"Hello, and welcome to Wikipedia! We welcome an...",0.0,290.0
3,57707.0,"Yes, and this page is wayyyyy too long as well...",1.0,181.0
4,10489.0,And when another Cfd inhabitant asked why the ...,0.0,401.0
...,...,...,...,...
495,79063.0,Hey now!! Nice. One never know what might happ...,0.0,92.0
496,156489.0,now that he's blocked me so that I can't respo...,1.0,60.0
497,61493.0,Is this Bugs person a particularly unsavoury t...,0.0,333.0
498,24773.0,"Er, what is the precise opposite of what the r...",1.0,273.0


## Токинезация и эмбеддинг

Загружу токенизатор и модель уже предтренерованные

In [452]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [453]:
tokenizer

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

Создам маску и паддинг

In [454]:
tokenized = df_tweets['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [455]:
pd.DataFrame(padded)

,0,1,2,3,4,5,6,7,8,9,...,234,235,236,237,238,239,240,241,242,243
0,101,2202,1037,2298,2012,1996,3931,2085,1011,2019,...,0,0,0,0,0,0,0,0,0,0
1,101,1000,1059,14341,19570,2389,1029,2245,2008,1000,...,0,0,0,0,0,0,0,0,0,0
2,101,7592,1010,1998,6160,2000,16948,999,2057,6160,...,0,0,0,0,0,0,0,0,0,0
3,101,2748,1010,1998,2023,3931,2003,2126,2100,2100,...,0,0,0,0,0,0,0,0,0,0
4,101,1998,2043,2178,12935,2094,21490,4630,2356,2339,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,101,4931,2085,999,999,3835,1012,2028,2196,2113,...,0,0,0,0,0,0,0,0,0,0
496,101,2085,2008,2002,1005,1055,8534,2033,2061,2008,...,0,0,0,0,0,0,0,0,0,0
497,101,2003,2023,12883,2711,1037,3391,4895,3736,6767,...,0,0,0,0,0,0,0,0,0,0
498,101,9413,1010,2054,2003,1996,10480,4500,1997,2054,...,0,0,0,0,0,0,0,0,0,0


Теперь сгенерирую Эмбэддинги и запишу их в ДФ features

In [456]:
batch_size = 100
embeddings = []
for i in range(padded.shape[0] // batch_size):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

In [457]:
features = np.concatenate(embeddings)

In [458]:
f=pd.DataFrame(features)
f=f.dropna()
f

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.123755,-0.106025,0.337740,-0.209775,-0.259691,-0.399717,0.476832,0.337581,0.316370,-0.475825,...,0.184848,-0.117560,0.025740,-0.741329,0.044063,0.250636,-0.506035,-0.116799,0.353760,0.585995
1,-0.187295,-0.267015,-0.260842,0.106950,-0.020278,0.010692,0.008199,0.091521,-0.088589,0.098095,...,0.077596,-0.042208,0.031694,0.042395,0.070803,0.027808,-0.409127,-0.266150,0.576978,0.549775
2,-0.130140,-0.339838,0.083788,0.051761,-0.311028,-0.825008,0.509609,0.593285,0.067492,-0.326785,...,0.180445,-0.139543,-0.299203,-0.366841,0.519180,0.239429,-0.295304,-0.197659,0.615162,0.686166
3,0.040030,0.099532,0.010963,-0.033622,-0.364336,-0.421667,0.191631,0.781923,0.185273,-0.130633,...,-0.079110,-0.087618,-0.095687,-0.348013,-0.075865,0.017607,-0.288373,-0.164290,0.400713,0.689111
4,0.048188,0.030478,0.230015,-0.007081,-0.141841,-0.301290,0.092908,0.752162,-0.099184,-0.271400,...,-0.000491,-0.040868,-0.012738,0.058175,0.262360,0.195151,-0.508119,-0.090298,0.524383,0.615540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.183561,-0.203617,0.223840,-0.086837,-0.386004,-0.449291,0.648740,0.846546,0.195980,-0.905144,...,-0.365974,-0.084881,0.061610,-0.051828,0.101859,-0.148377,-0.151947,0.037575,0.368775,0.326369
496,-0.037299,0.260283,-0.016207,-0.015196,-0.192107,-0.243083,0.132104,0.241418,-0.162050,-0.318341,...,0.045470,-0.259458,0.070598,-0.143383,0.069786,0.022542,0.008019,-0.036899,0.206848,0.412416
497,0.026393,-0.053288,0.142881,-0.146705,-0.247270,-0.265700,0.479407,0.415640,-0.006921,-0.211179,...,0.101757,0.038214,-0.012887,-0.390732,0.031235,0.253130,-0.082880,-0.028364,0.551593,0.745340
498,0.137219,0.050578,0.010386,-0.373457,-0.366369,-0.450904,0.567493,0.499609,0.126013,-0.259215,...,-0.006776,-0.074343,-0.072544,-0.522449,0.323080,0.201090,-0.273104,-0.391786,0.487330,0.661489


## Разбиение датасета на тест и трейн

In [459]:
df=pd.DataFrame(f)
df['y']=df_tweets['toxic']
X=df.drop('y',axis=1)
Y=df_tweets['toxic']

In [460]:
Y


0      1.0
1      0.0
2      0.0
3      1.0
4      0.0
      ... 
495    0.0
496    1.0
497    0.0
498    1.0
499    1.0
Name: toxic, Length: 500, dtype: float64

In [461]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.25)

In [462]:
Y_train

230    0.0
35     0.0
246    0.0
53     0.0
150    1.0
      ... 
411    1.0
200    1.0
484    1.0
371    1.0
262    1.0
Name: toxic, Length: 375, dtype: float64

## Подбор гипперпараметров и обучение моделей

### Логистическая регрессия

In [481]:
"""Fitting 5 folds for each of 28 candidates, totalling 140 fits
0.8307465159702729 {'C': 0.005999999999999998, 'max_iter': 900}"""
param_grid = {'C': np.arange(0.0045, 0.0065, 0.0005),
              'max_iter':range(900, 1250, 50),
             }
grd_rf = GridSearchCV(LogisticRegression(random_state=0),
                            param_grid=param_grid,cv=5,
                            scoring='f1', n_jobs=-1,verbose=3)
grd_rf.fit(X_train,Y_train)
print(grd_rf.best_score_,grd_rf.best_params_)

Fitting 5 folds for each of 28 candidates, totalling 140 fits
0.8307465159702729 {'C': 0.005999999999999998, 'max_iter': 900}


In [482]:
lr=LogisticRegression(C=0.006,max_iter=900,random_state=0)
lr.fit(X_train,Y_train)
f1_score(Y_test,lr.predict(X_test))

0.8428571428571429

In [483]:
preds=pd.DataFrame(Y_test)
preds['pred']=lr.predict(X_test)

In [494]:
preds

,toxic,pred,cat
111,1.0,1.0,1.0
241,1.0,1.0,1.0
180,1.0,0.0,0.0
295,1.0,1.0,1.0
448,1.0,0.0,0.0
...,...,...,...
273,1.0,0.0,0.0
487,0.0,0.0,0.0
223,1.0,1.0,1.0
245,0.0,0.0,0.0


### CatBoost

In [467]:
cat=CatBoostClassifier(iterations=4000,
                           depth=2,
                           learning_rate=0.001,
                           loss_function='Logloss')

In [ ]:
"""0.8444624624624625 {'depth': 5, 'iterations': 500, 'learning_rate': 0.05} 2.5 часа)"""
param_grid2 = {'depth': [1, 2, 5],
              'iterations':range(500, 2000, 500),
              'learning_rate':[0.001,0.0001,0.05]
             }
grd_rf2 = GridSearchCV(CatBoostClassifier(),
                            param_grid=param_grid2,cv=5,
                            scoring='f1', n_jobs=-1,verbose=1)
grd_rf2.fit(X_train,Y_train)
print(grd_rf2.best_score_,grd_rf2.best_params_)

In [472]:
grd_rf2.best_estimator_.fit(X_train,Y_train,verbose=100)

0:	learn: 0.6699305	total: 424ms	remaining: 3m 31s
100:	learn: 0.1041691	total: 16.8s	remaining: 1m 6s
200:	learn: 0.0218770	total: 38.2s	remaining: 56.8s
300:	learn: 0.0078040	total: 49.9s	remaining: 33s
400:	learn: 0.0041360	total: 1m 4s	remaining: 15.9s
499:	learn: 0.0027318	total: 1m 18s	remaining: 0us


In [473]:
f1_score(Y_test,grd_rf2.best_estimator_.predict(X_test))

0.8358208955223881

В целом f1 = 0.835 хороший результат.

In [485]:
preds['cat']=grd_rf2.best_estimator_.predict(X_test)

In [493]:
preds

,toxic,pred,cat
111,1.0,1.0,1.0
241,1.0,1.0,1.0
180,1.0,0.0,0.0
295,1.0,1.0,1.0
448,1.0,0.0,0.0
...,...,...,...
273,1.0,0.0,0.0
487,0.0,0.0,0.0
223,1.0,1.0,1.0
245,0.0,0.0,0.0


### Лес

In [498]:
rf=RandomForestClassifier(random_state=42)

In [ ]:
rf.fit(X_train,Y_train)
f1_score(Y_test,rf.predict(X_test))

In [511]:
param_grid = {'n_estimators': range(120, 200, 5),
              'max_depth':np.arange(1, 8, 1),
              'min_samples_split':range(2, 4),
              'min_samples_leaf':range(1, 3)
             }
grd_rf3 = RandomizedSearchCV(rf, param_distributions=param_grid,cv=5, scoring='f1', n_jobs=-1,random_state=42)

grd_rf3.fit(X_train,Y_train)
print(grd_rf3.best_score_,grd_rf3.best_params_)

0.8261195400632021 {'n_estimators': 120, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 4}


In [516]:
grd_rf3.best_estimator_.fit(X_train,Y_train)
f1_score(Y_test,grd_rf3.best_estimator_.predict(X_test))

0.8088235294117647

In [517]:
preds['forest']=grd_rf3.best_estimator_.predict(X_test)

In [533]:
preds=preds.reset_index(drop=True)
preds

,toxic,pred,cat,forest
0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0
2,1.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0
4,1.0,0.0,0.0,0.0
...,...,...,...,...
120,1.0,0.0,0.0,0.0
121,0.0,0.0,0.0,0.0
122,1.0,1.0,1.0,1.0
123,0.0,0.0,0.0,0.0


### Тест

Здесь я закинул результаты трех моделей в ДФ и по ним пропустить линейную регрессию, и получить более полный результат (да я здесь сделал из тестовой выборки еще выборки, но я думаю для эксперимента хватати, это проще чем заново все переобучать на новой выборке большего размера)

In [534]:
Y_ens=preds['toxic']
X_ens=preds.drop('toxic',axis=1)

In [535]:
Y_ens.isna().mean()

0.0

In [536]:
X_train_en,X_test_en,Y_train_en,Y_test_en=train_test_split(X_ens,Y_ens,test_size=0.25)

In [537]:
lin_regr=LinearRegression()
lin_regr.fit(X_train_en,Y_train_en)
pred_tot=lin_regr.predict(X_test_en)


In [539]:
pred_tot

array([0.93585418, 0.93585418, 0.20756949, 0.93585418, 0.20756949,
       0.20756949, 0.93585418, 0.93585418, 0.6941246 , 0.93585418,
       0.93585418, 0.20756949, 0.93585418, 0.20756949, 1.24566042,
       0.93585418, 0.93585418, 0.20756949, 0.93585418, 0.20756949,
       0.93585418, 0.93585418, 0.93585418, 0.93585418, 0.20756949,
       0.93585418, 0.20756949, 0.20756949, 0.93585418, 0.93585418,
       0.44929906, 0.93585418])

In [546]:
test_pred=pd.DataFrame(Y_test_en)
test_pred=test_pred.reset_index(drop=True)
test_pred['tot']=pred_tot
test_pred.head(5)

,toxic,tot
0,1.0,0.935854
1,1.0,0.935854
2,0.0,0.207569
3,1.0,0.935854
4,0.0,0.207569


In [552]:
X_test_en=X_test_en.reset_index(drop=True)
test_pred['sum']=X_test_en['pred']+X_test_en['cat']+X_test_en['forest']
test_pred.head(5)

,toxic,tot,sum,goal
0,1.0,0.935854,3.0,1.0
1,1.0,0.935854,3.0,NaN
2,0.0,0.207569,0.0,NaN
3,1.0,0.935854,3.0,NaN
4,0.0,0.207569,0.0,NaN


In [556]:
for i in range(len(test_pred)):
  if test_pred.loc[i,'tot']>0.5:
    test_pred.loc[i,'goal']=1
  else:
    test_pred.loc[i,'goal']=0

  if test_pred.loc[i,'sum']>=2:
    test_pred.loc[i,'sum_goal']=1
  else:
    test_pred.loc[i,'sum_goal']=0
test_pred.head()

,toxic,tot,sum,goal,sum_goal
0,1.0,0.935854,3.0,1.0,1.0
1,1.0,0.935854,3.0,1.0,1.0
2,0.0,0.207569,0.0,0.0,0.0
3,1.0,0.935854,3.0,1.0,1.0
4,0.0,0.207569,0.0,0.0,0.0


In [544]:
f1_score(Y_test_en,test_pred['goal'])#Линейная регрессия

0.8421052631578947

In [557]:
f1_score(Y_test_en,test_pred['sum_goal'])#Сумма голосования по 2 из 3

0.7894736842105262

# Вывод

По итогу выполнения проекта, были получены эмбеддинги от BERT, на основе которых были созданы модели обучения предсказания токсичности комментариев.

f1 score Логистической Регресси на тесте 0.8428

f1 score CatBoost на тесте 0.8358

f1 score RandomForest на тесте 0.808

Получилось иронично, я вероятно плохо угадал с диапазоном подбора гиппер параметров для CatBoost, и он за 2,5 часа выдал скор меньше ЛогРегрессии, обучающейся за секунды) Думаю как вариант из Кота можно выжить больше. Либо разбить выборки на 3 части, и построить ансамбль на результатах ЛогРегрессии и Кота на основе ЛинРегрессии, и возможно получить бОльший скор.

Провел тест по предикту по 3 предиктам,
получился результат f1 score 0.842.
В целом это результат схожий с ЛогРегрессией и выхлопа нет. НО если бы выборка была не 125 объектов, то мб это сработало лучше. Так что наверное скорее хорошо, чем плохо)